In [1]:
import numpy as np
import pandas as pd
import descartes
# import geopandas as gpd
# from shapely.geometry import Point, Polygon

In [2]:
df = pd.read_csv('ghcnd_hcn/pair_final.csv')
df.shape

(524065, 15)

In [3]:
test_df = df.query("STATION == 'USC00057936'")
lat, lon = test_df.LAT.values[0], test_df.LON.values[0]
test_df.shape

(5506, 15)

In [14]:
nearby_station = df[(lat-1 < df.LAT ) &
   (df.LAT < lat+1) & 
  (lon-1 < df.LON ) &
   (df.LON < lon+1)
  ]

nearby_station['STATION'].unique()

# nearby_station.drop(
#     df.index[train_df.query("STATION == 'USC00057936'").index])

array(['USC00052281', 'USC00057936'], dtype=object)

In [48]:
distance_arr = []
test_df = df.query("STATION == 'USC00057936'")
test_location = np.array([test_df[element].values[0] for element in ['LON', 'LAT', 'ELEV']])

k = 3

for station_id, station_name in enumerate(df['STATION'].unique()):
    nearby_df = df.query("STATION == '{}'".format(station_name))
    nearby_location = np.array([nearby_df[element].values[0] for element in ['LON', 'LAT', 'ELEV']])
    nearby_distance = np.linalg.norm(test_location - nearby_location)
    
    distance_arr.append(nearby_distance)

k_nearest_stations = df['STATION'].unique()[np.argpartition(distance_arr, k)[:k]]

In [49]:
nearby_stations_df = df.query("STATION in @k_nearest_stations")
nearby_stations_df

,STATION,LAT,LON,ELEV,DATE,PRCP1,TMAX1,TMIN1,SNOW1,SNOWD1,PRCP2,TMAX2,TMIN2,SNOW2,SNOWD2
97380,USC00023596,36.0528,-112.1503,2068.1,722815.0,0.0,44.0,-61.0,0.0,102.0,0.0,44.0,-61.0,0.0,102.0
97381,USC00023596,36.0528,-112.1503,2068.1,722817.0,0.0,44.0,-61.0,0.0,102.0,0.0,44.0,-61.0,0.0,102.0
97382,USC00023596,36.0528,-112.1503,2068.1,722819.0,0.0,44.0,-61.0,0.0,102.0,0.0,44.0,-61.0,0.0,102.0
97383,USC00023596,36.0528,-112.1503,2068.1,722821.0,0.0,44.0,-61.0,0.0,102.0,0.0,44.0,-61.0,0.0,102.0
97384,USC00023596,36.0528,-112.1503,2068.1,722823.0,0.0,44.0,-61.0,0.0,102.0,0.0,44.0,-61.0,0.0,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326511,USC00057936,40.4883,-106.8233,2092.8,737323.0,0.0,306.0,50.0,0.0,0.0,0.0,306.0,50.0,0.0,0.0
326512,USC00057936,40.4883,-106.8233,2092.8,737325.0,0.0,306.0,50.0,0.0,0.0,0.0,306.0,50.0,0.0,0.0
326513,USC00057936,40.4883,-106.8233,2092.8,737327.0,0.0,306.0,50.0,0.0,0.0,0.0,306.0,50.0,0.0,0.0
326514,USC00057936,40.4883,-106.8233,2092.8,737329.0,0.0,306.0,50.0,0.0,0.0,0.0,306.0,50.0,0.0,0.0


In [50]:
nearby_stations_df

,STATION,LAT,LON,ELEV,DATE,PRCP1,TMAX1,TMIN1,SNOW1,SNOWD1,PRCP2,TMAX2,TMIN2,SNOW2,SNOWD2
97380,USC00023596,36.0528,-112.1503,2068.1,722815.0,0.0,44.0,-61.0,0.0,102.0,0.0,44.0,-61.0,0.0,102.0
97381,USC00023596,36.0528,-112.1503,2068.1,722817.0,0.0,44.0,-61.0,0.0,102.0,0.0,44.0,-61.0,0.0,102.0
97382,USC00023596,36.0528,-112.1503,2068.1,722819.0,0.0,44.0,-61.0,0.0,102.0,0.0,44.0,-61.0,0.0,102.0
97383,USC00023596,36.0528,-112.1503,2068.1,722821.0,0.0,44.0,-61.0,0.0,102.0,0.0,44.0,-61.0,0.0,102.0
97384,USC00023596,36.0528,-112.1503,2068.1,722823.0,0.0,44.0,-61.0,0.0,102.0,0.0,44.0,-61.0,0.0,102.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326511,USC00057936,40.4883,-106.8233,2092.8,737323.0,0.0,306.0,50.0,0.0,0.0,0.0,306.0,50.0,0.0,0.0
326512,USC00057936,40.4883,-106.8233,2092.8,737325.0,0.0,306.0,50.0,0.0,0.0,0.0,306.0,50.0,0.0,0.0
326513,USC00057936,40.4883,-106.8233,2092.8,737327.0,0.0,306.0,50.0,0.0,0.0,0.0,306.0,50.0,0.0,0.0
326514,USC00057936,40.4883,-106.8233,2092.8,737329.0,0.0,306.0,50.0,0.0,0.0,0.0,306.0,50.0,0.0,0.0
